In [2]:
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus.utils import config
import pandas as pd
import re

pd.set_option('display.max_columns', None)

#config['Authentication']['APIKey'] = 'cc5139cb362dbf0aac149aabdfa4011f'   ###eduroam 사용해서만 이용 가능.
config['Authentication']['InstToken'] = ''

def keep_letters_and_digits(input_string):
    #  영어 알파벳(a-zA-Z)과 숫자(0-9)를 제외한 모든 문자를 찾아서 제거.
    result = re.sub(r'[^a-zA-Z0-9\'\,\.óàáâãäåçèéêëìíîïñòóôõöùúûü]', ' ', input_string)
    result = result.strip()
    return result

best_papers = pd.read_csv("./data/best_paper_awards_crawl.csv")

### API 통해 해당 Paper 정보 가져오기
- Eduroam 에서만 사용 가능

In [3]:
only_first = best_papers.groupby(['year', 'paper']).first().reset_index()

total_set = pd.DataFrame()
lost_set = pd.DataFrame()
not_done = []

for row in only_first.iterrows() :
    paper = (row[1]['paper'].replace("(", "").replace(")", ""))
    paper = keep_letters_and_digits(paper)
    paper = paper.replace("  ", " ")
    author = (row[1]['author'].split(" "))
    if author[len(author)-1] != '': 
        author = author[len(author)-1]
    else :
        author = author[len(author)-2]

    out = ' TITLE-ABS-KEY ( ' + paper + ' ) AND AUTH ( ' + author + ' )' 

    s = ScopusSearch(out, download=True, # 검색 결과를 저장합니다. 
                    verbose=False)  # 진행 상황을 표시합니다.

    if s.get_results_size() == 0 :
        if author != keep_letters_and_digits(author) :
            out = ' TITLE ( ' + paper + ' )'
            s = ScopusSearch(out, download=True, # 검색 결과를 저장합니다. 
                verbose=False)  # 진행 상황을 표시합니다.
            if s.get_results_size() != 0 :
                df_s = pd.DataFrame(s.results)
                df_s = df_s.sort_values('citedby_count',ascending=False).head(1)
                df_s['origin_ref'] = row[1]['conference']
                total_set = pd.concat([total_set, df_s])
                print(out, " -> Success")
                continue
        
        paper = paper.replace(" and ", " ")
        paper = paper.replace(" or ", " ")
        paper = paper.replace(" ", " AND ")

        out = ' TITLE ( ' + paper + ' ) '
        s = ScopusSearch(out, download=True, # 검색 결과를 저장합니다. 
                    verbose=False)  # 진행 상황을 표시합니다.
        
        if s.get_results_size() != 0 :
            df_s = pd.DataFrame(s.results)
            df_s['origin_ref'] = row[1]['conference']
            df_s = df_s[df_s['coverDate'].str.startswith(str(row[1]['year']))]
            df_s = df_s.sort_values('citedby_count',ascending=False).head(1)
            total_set = pd.concat([total_set, df_s])
            print(out, " -> Success")
            continue
        else :
            not_done.append(row[1])
            print(out, " -> No results")
    elif s.get_results_size() == 1:
        df_s = pd.DataFrame(s.results)
        df_s['origin_ref'] = row[1]['conference']
        total_set = pd.concat([total_set, df_s])
        print(out, " -> Success")
    else :
        df_s = pd.DataFrame(s.results)
        if len(df_s[df_s['coverDate'].str.startswith(str(row[1]['year']))]) != 0 :
            df_s = df_s[df_s['coverDate'].str.startswith(str(row[1]['year']))]
            if len(df_s.loc[df_s['subtype']=='cp']) != 0 :
                df_s = df_s.loc[df_s['subtype']=='cp']

        elif len(df_s.loc[df_s['subtype']=='cp']) != 0 :
            df_s = df_s.loc[df_s['subtype']=='cp']

        df_s = df_s.sort_values('citedby_count',ascending=False).head(1)
        df_s['origin_ref'] = row[1]['conference']
        total_set = pd.concat([total_set, df_s])
        print(out, " -> Success")


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


 TITLE-ABS-KEY ( A Novel Application of Theory Refinement to Student Modeling ) AND AUTH ( Baffes )  -> Success
 TITLE-ABS-KEY ( An algorithm for multicast tree generation in networks with asymmetric links ) AND AUTH ( Ramanathan )  -> Success
 TITLE-ABS-KEY ( Automatic Compiler Inserted I O Prefetching for Out of Core Applications ) AND AUTH ( Mowry )  -> Success
 TITLE-ABS-KEY ( Exploiting Process Lifetime Distributions for Dynamic Load Balancing ) AND AUTH ( Harchol-Balter )  -> Success
 TITLE-ABS-KEY ( Fast Subsequence Matching in Time Series Databases ) AND AUTH ( Faloutsos )  -> Success
 TITLE-ABS-KEY ( Implementing data cubes efficiently ) AND AUTH ( Harinarayan )  -> Success
 TITLE-ABS-KEY ( Indigo A Local Propagation Algorithm for Inequality Constraints ) AND AUTH ( Borning )  -> Success
 TITLE-ABS-KEY ( Pushing the Envelope Planning, Propositional Logic, and Stochastic Search ) AND AUTH ( Kautz )  -> Success
 TITLE-ABS-KEY ( Retrieving spoken documents by combining multiple i

In [4]:
print("전체 불러온 Paper 수:", len(total_set))
print("불러오지 못한 Paper 수:", len(not_done))

전체 불러온 Paper 수: 1464
불러오지 못한 Paper 수: 42


In [9]:
total_set.to_csv("./data/best_scopus.csv", index = False)

In [8]:
total_set.groupby('origin_ref').count().reset_index()[['origin_ref', 'eid']].sort_values('eid', ascending= False)

,origin_ref,eid
2,CHI Human-Computer Interaction,326
9,ICSE Software Engineering,143
6,FSE Software Engineering,103
18,PLDI Programming Languages,63
29,UIST User Interface,47
0,AAAI Artificial Intelligence,43
10,IJCAI Artificial Intelligence,42
8,ICML Machine Learning,41
28,STOC Theory,40
27,SOSP Operating Systems,40


In [10]:
pd.DataFrame(not_done).groupby('year')['paper'].count().reset_index()

,year,paper
0,1996,1
1,1999,1
2,2001,2
3,2006,1
4,2010,3
5,2012,1
6,2013,1
7,2014,3
8,2015,1
9,2016,2


In [62]:
total_set['author_count'] = total_set['author_count'].astype(int)
print("논문 별 평균 저자 수",total_set['author_count'].mean())

df_exploded = total_set.assign(affiliation_country=total_set['affiliation_country'].str.split(';')).explode('affiliation_country')
print("논문 별 평균 공동 작업 국가 수", df_exploded.groupby('eid')['affiliation_country'].nunique().mean())

논문 별 평균 저자 수 4.12901023890785
논문 별 평균 공동 작업 국가 수 1.3766233766233766


In [63]:
###공동작업 국가수, Paper수
#Affiliation 정보가 없는 경우, 공동작업 국가 수가 0
df_exploded.groupby('eid')['affiliation_country'].nunique().reset_index().groupby('affiliation_country').count().reset_index()

,affiliation_country,eid
0,0,3
1,1,1027
2,2,329
3,3,89
4,4,13
5,5,2


In [64]:
###국가별 저자 수
df_exploded.groupby("affiliation_country")['eid'].count().reset_index().sort_values('eid', ascending= False)

,affiliation_country,eid
48,United States,1498
47,United Kingdom,198
8,China,149
14,Germany,140
6,Canada,140
13,France,76
43,Switzerland,66
1,Australia,59
21,Israel,48
16,Hong Kong,42


### Affilication과 Co-Author 가져오기

In [15]:
from pybliometrics.scopus import AuthorRetrieval
from pybliometrics.scopus import AffiliationRetrieval
au2 = AuthorRetrieval(57207594975)
au2.coauthor_link
aff1 = AffiliationRetrieval(au2.affiliation_current[0].id)

In [16]:
print(aff1.country)
print(aff1.city)
print(aff1.address)
print(aff1.affiliation_name)

United States
Boston
233 Bay State Road
Boston University


In [17]:
coauthors = pd.DataFrame(au2.get_coauthors())
coauthors.head()

,surname,given_name,id,areas,affiliation_id,name,city,country
0,Schwan,Karsten,7006465424,Computer Science (all); Engineering (all); Mat...,60019647,Georgia Institute of Technology,Atlanta,United States
1,Grant,Patricia Ellen,7401614401,Medicine (all); Neuroscience (all); Computer S...,60002746,Harvard Medical School,Boston,United States
2,Grbic,Anthony,6602191591,Physics and Astronomy (all); Engineering (all)...,60025778,"University of Michigan, Ann Arbor",Ann Arbor,United States
3,Žilić,Željko,6701462755,Computer Science (all); Engineering (all); Mat...,60002494,Université McGill,Montreal,Canada
4,Leeser,Miriam E.,7004246699,Computer Science (all); Engineering (all); Mat...,60141202,College of Engineering,Boston,United States
